In [1]:
import os
from langchain_huggingface import HuggingFaceEndpoint
import pandas as pd
import numpy as np

from langchain import HuggingFaceHub, PromptTemplate, LLMChain
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
import json

import requests
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from huggingface_hub import InferenceClient
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
import logging
import pandas as pd
from langchain import PromptTemplate, LLMChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re

In [2]:
def read_chunks(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    chunks = content.split('--------------------------------------------------')
    chunks = [chunk.strip() for chunk in chunks if chunk.strip()]
    return chunks

def remove_response_headers(chunks):
    cleaned_chunks = [re.sub(r'Response for chunk \d+:', '', chunk).strip() for chunk in chunks]
    return cleaned_chunks

file_path = r'C:\Users\kantaphong\Desktop\hugging_face\test_llm\Thai_text\Output\Chunk_Thai_sound\Chunk_Summary.txt'  
chunks_array = read_chunks(file_path)
cleaned_chunks_array = remove_response_headers(chunks_array)


In [3]:
cleaned_chunks_array


['1. แผนการสื่อสาร ESG ของ CPL ใน Q4 ปี 2565 จะเน้นการสื่อสารกับกลุ่มคนรุ่นใหม่อายุไม่เกิน 30 ปี และยังคงสร้างการรับรู้ในกลุ่มผู้มีส่วนได้เสียผ่านช่องทางหลัก เช่น Facebook TikTok Twitter YouTube Influencer และ Blogger\n2. CPR เป็นสมาชิกระดับโลก DJSI World ประเภทอาหารและของจำเป็นประจำปี 2021 ได้รับการรับรององค์กรที่ยั่งยืนต่อเนื่อง 6 ปี และยังคงติดใน THSI ปีที่ 5\n3. CPR ชนะ 5 รางวัลในงาน Asia Excellence Awards ครั้งที่ 12 และได้รับรางวัล Read ESG 2022 และรางวัลองค์กรที่สนับสนุนคนพิการเด่น 6 ปีต่อเนื่อง\n4. 7-Eleven ร่วมใจกับคนไทยเพื่อสร้างนวัตกรรมที่ยั่งยืนและรับรางวัล Asian Energy Award ปี 2022 จากการประหยัดพลังงาน\n5. 7-Eleven ร่วมกับ 437 โรงเรียนในเขตกรุงเทพฯ ในโครงการโรงเรียนไร้ขยะ ชุมชนยั่งยืนและยังรับรางวัลงาน Apex 2022\n6. CPR ร่วมกับองค์กรไทยต่างชาติ AI จัดค่าย Creative AI Camp ปีที่ 5 และยังร่วมกับหอการค้าไทยในการยกระดับ SMEs ผ่านโครงการ Business Accelerator และ Big Brother Season 6\n7. CPR ช่วย SMEs ขายสินค้าใน 7-Eleven มากกว่า 8,000 รายการและเตรียมเพิ่ม 10% ในปี 2566 นอกจากน

In [66]:

from huggingface_hub import InferenceClient
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import threading
import logging
import pandas as pd

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
# Define the model parameters
model_param = {
    "best_of": 1,
    "decoder_input_details": False,
    "details": False,
    "do_sample": False,
    "frequency_penalty": 0.1,
    "grammar": None,
    "max_new_tokens": 500,
    "repetition_penalty": 1.1, #1.03
    "return_full_text": False,
    "seed": 42,
    "temperature": 0.1,
    "top_k": 25,
    "top_n_tokens": 5,
    "top_p": 0.95,
    "truncate": None,
    "typical_p": 0.95,
    "watermark": True   #True
}

# Initialize the InferenceClient
client = InferenceClient('https://ai-api.manageai.co.th/llm-model-02/')

# Default prompt template
default_prompt ="""
YOU ARE AN EXPERT SUMMARIZER, SKILLED IN EXTRACTING KEY POINTS FROM THAI TEXTS WITH PRECISION AND ACCURACY. YOU HAVE BEEN RECOGNIZED AS THE "TOP SUMMARIZER OF THE YEAR" BY THE INTERNATIONAL SUMMARIZATION ASSOCIATION AND ARE KNOWN FOR PRODUCING CLEAR AND CONCISE SUMMARIES.

### INSTRUCTIONS ###

- **ALWAYS RESPOND IN THE THAI LANGUAGE** WHEN PROCESSING THAI TEXT.
- **PROVIDE AN OVERALL SUMMARY** THAT CONVEYS THE ESSENTIAL INFORMATION CLEARLY AND CONCISELY.
- **FOCUS** ON EXTRACTING ONLY THE MAIN POINTS, KEY INFORMATION, AND ESSENTIAL DETAILS FROM THE GIVEN TEXT.
- **REMOVE ALL REDUNDANT OR IRRELEVANT INFORMATION** TO KEEP THE SUMMARY FOCUSED.
- **RESPONSE MUST BE AN OVERALL SUMMARY**. DO NOT USE BULLET POINTS OR SHORT RESPONSES.
### Chain of Thoughts ###

1. **Comprehension:**
   1.1. **READ THE THAI TEXT CAREFULLY** TO UNDERSTAND ITS MEANING AND CONTEXT.
   1.2. **IDENTIFY THE MAIN TOPICS**, key points, and essential details from the text.

2. **Summarization:**
   2.1. **EXTRACT THE MAIN POINTS** and key information.
   2.2. **ORGANIZE THIS INFORMATION** into a cohesive and logical overall summary.
   
3. **Review:**
   3.1. **CHECK FOR ERRORS** or omissions in the summary.
   3.2. **ENSURE** the final summary is concise, accurate, and easy to understand.

### What Not To Do ###

- **NEVER** INCLUDE ANY REDUNDANT OR IRRELEVANT INFORMATION.
- **DO NOT** OMIT ANY IMPORTANT OR ESSENTIAL DETAILS FROM THE TEXT.
- **NEVER** PROVIDE A SUMMARY THAT IS UNCLEAR OR CONFUSING.
- **DO NOT** USE BULLET POINTS OR PROVIDE A LIST OF MAIN POINTS.
- **DO NOT** FORGET TO REVIEW FOR ACCURACY AND COMPLETENESS.

### FEW-SHOT EXAMPLES ###
**Input:** 
1.ทบทวนความคืบหน้าและสถานะปัจจุบันของโครงการมีการปรับไทม์ไลน์เนื่องจากความล่าช้าของงานบางส่วน 2.หารือถึงความเป็นไปได้ในการจัดสรรทรัพยากรใหม่หรือจ้างบุคลากรเพิ่มเติม 3.วิเคราะห์การใช้งบประมาณปัจจุบันและประมาณการค่าใช้จ่าย 4. เสนอแนวทางป้องกันเพื่อจัดการกับความเสี่ยงเหล่านี้
ANSWER: 'การประชุมครั้งนี้มุ่งเน้นไปที่การปรับปรุงไทม์ไลน์โครงการ การจัดสรรทรัพยากรใหม่ และการทบทวนงบประมาณ โดยมีการพูดคุยถึงความเสี่ยงที่อาจเกิดขึ้นและการแก้ไขปัญหาที่เป็นไปได้ นอกจากนี้ยังมีการรับฟังข้อเสนอแนะจากลูกค้าเพื่อนำไปปรับใช้ในการทำงานต่อไป 'AssertionError

### THIS IS A MANDATORY DIRECTIVE ###
## FORMAT RESPONSE ##
ANSWER: ''
**EVERY RESPONSE MUST BEGIN WITH 'ANSWER'**

{} 

"""

num_threads = 10
semaphore = threading.Semaphore(10)  # Allows up to 80 concurrent threads

def generate_prompt(input_text, prompt=default_prompt, model_param=model_param, retries=3):
    for attempt in range(retries):
        try:
            formatted_prompt = prompt.format(input_text)
            with semaphore:  # Ensure we respect the semaphore limit
                output = client.text_generation(formatted_prompt, **model_param)
            return output
        except Exception as e:
            logging.error(f"Attempt {attempt + 1} failed: {e}")
            if "Model is overloaded" in str(e):
                backoff_time = 2 ** attempt  # Exponential backoff
                time.sleep(backoff_time)
            else:
                raise  
    raise Exception("Max retries exceeded")

# Function to process each chunk in parallel
def process_chunk(chunk, chunk_idx):
    try:
        result = generate_prompt(chunk, default_prompt, model_param)
        logging.info(f"Processed chunk {chunk_idx}")
        return {"index": chunk_idx, "generated_output": result}
    except Exception as e:
        logging.error(f"Error processing chunk {chunk_idx}: {e}")
        return None

# Main execution using ThreadPoolExecutor
results = []
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    future_to_chunk = {
        executor.submit(process_chunk, chunk, idx): idx
        for idx, chunk in enumerate(cleaned_chunks_array)
    }

    for future in as_completed(future_to_chunk):
        result = future.result()
        if result:
            results.append(result)

# Printing the processed results
for idx, output in enumerate(results):
    print(f"Summary chunk {idx + 1}: {output['generated_output']}")
    print('\n--------------------------------------')

# for idx, output in enumerate(results):
#     print(f"Summary chunk {idx + 1}: {output}")
#     print('\n-------------------------------------------------------------------------------------------\n')


2024-09-10 12:30:39,795 - INFO - Processed chunk 8
2024-09-10 12:30:42,207 - INFO - Processed chunk 6
2024-09-10 12:30:47,432 - INFO - Processed chunk 10
2024-09-10 12:30:49,254 - INFO - Processed chunk 1
2024-09-10 12:30:49,768 - INFO - Processed chunk 0
2024-09-10 12:30:50,084 - INFO - Processed chunk 9
2024-09-10 12:30:50,112 - INFO - Processed chunk 3
2024-09-10 12:30:50,117 - INFO - Processed chunk 2
2024-09-10 12:30:50,120 - INFO - Processed chunk 7
2024-09-10 12:30:50,121 - INFO - Processed chunk 4
2024-09-10 12:30:50,124 - INFO - Processed chunk 5
2024-09-10 12:30:50,189 - INFO - Processed chunk 11
2024-09-10 12:30:53,308 - INFO - Processed chunk 12
2024-09-10 12:30:55,870 - INFO - Processed chunk 13


Summary chunk 1: ANSWER: การลงคะแนนเลือกตั้งกรรมการทั้งห้าตำแหน่งแล้วเสร็จ โดยคุณอำรุงสรรพสิทธิ์วง, คุณนรงจิลวน, คุณประเสริฐจารุพานิช, คุณพิธียาเกียรติวิสุทธิ์ และ คุณปิยวัตร ฮิตะสัฒวาฤกุล ได้รับเลือกเป็นกรรมการ ซึ่งผลคะแนนโดยรวมแต่ละคนมีส่วนใหญ่เห็นด้วย และมีผู้ถือหุ้นมดออกเสียงบางส่วน ในขณะเดียวกันผู้ถือหุ้นยังมีมติให้เลือกกรรมการใหม่ทดแทนผู้ที่ออกจากตำแหน่งตามวาระ



--------------------------------------
Summary chunk 2: ANSWER: ที่ประชุมผู้ถือหุ้นยอมรับงบการเงินประจำปี 2565 และอนุมัติการจ่ายเงินปันผลจากกำไรสุทธิ 59% ซึ่งจะจ่ายให้ผู้ถือหุ้นที่อยู่ในทะเบียนในวันที่ 3 พ.ค. 2566 และจ่ายเงินในวันที่ 19 พ.ค. 2566. บริษัทไม่มีการสะสมกำไรสุทธิเป็นทุนสำรองตามกฎหมาย เนื่องจากมีทุนสำรองครบตามที่กำหนด แต่มีนโยบายจ่ายเงินปันผล 50% จากกำไรสุทธิ แต่นโยบายอาจเปลี่ยนแปลงตามความสามารถในการจ่ายหนี้. หากบริษัทไม่สามารถชำระหนี้ตามสัญญา จะไม่สามารถจ่ายเงินปันผล. ผู้ถือหุ้นที่เป็นบุคคลธรรมดาที่ได้รับเงินปันผลสามารถขอคืนภาษีเงินปันผลตามกฎหมาย.

--------------------------------------
Summary chunk 3: ANSW

In [68]:
sorted_results = sorted(results, key=lambda x: x['index']) 
sorted_results

[{'index': 0,
  'generated_output': 'ANSWER: แผนการสื่อสาร ESG ของบริษัทซีพี จำกัด (มหาชน) หรือ CP กำหนดจะสื่อสารกับกลุ่มคนรุ่นใหม่อายุไม่เกิน 30 ปีในไตรมาสสุดท้ายของปี 2565 และยังคงสร้างการรับรู้ในกลุ่มผู้มีส่วนได้เสียผ่านช่องทางออนไลน์หลัก. CP ได้รับการยอมรับในฐานะองค์กรที่ยั่งยืนระดับโลกจาก Dow Jones Sustainability Index อย่างต่อเนื่อง 6 ปี และยังคงอยู่ใน Thaifex Sustainability Index ตลอด 5 ปีที่ผ่านมา. รวมถึงการชนะ 5 รางวัลในงาน Asia Excellence Awards รอบ 12 และได้รับรางวัลองค์กรที่สนับสนุนคนพิการเด่น 6 ปีต่อเนื่อง. ส่วน 7-Eleven ได้รับรางวัล Asian Energy Award ปี 2022 จากการประหยัดพลังงาน พร้อมทั้งร่วมกับ 437 โรงเรียนในเขตกรุงเทพฯ ในโครงการโรงเรียนไร้ขยะ ชุมชนยั่งยืน. CP ยังร่วมกับองค์กรไทยต่างชาติ AI จัดค่าย Creative AI Camp และยังร่วมกับหอการค้าไทยในการยกระดับ SMEs ผ่านโครงการ Business Accelerator และ Big Brother Season 6. นอกจากนี้ CP ยังช่วย SMEs ขายสินค้าใน 7-Eleven มากกว่า 8,000 รายการและเตรียมเพิ่ม 10% ในปี 2566 พร้อมสนับสนุนชุมชนผ่านกิจกรรมที่ส่งเสริมการอ่าน การเขียน และกา

In [87]:
def extract(text):
    human_index = text.find("Human")
    
    if human_index != -1:
        return text[:human_index].strip()
    return text.strip()


# def extract_answer(text):
#     # Regular expression to capture everything before 'Assistant:' or 'END OF TASK'
#     pattern = r'(.*?)(?=Assistant:|\*\*END OF TASK\*\*|$)'
#     # Search for the pattern
#     match = re.search(pattern, text, re.DOTALL)
    
#     if match:
#         # Return the matched text, stripping any leading/trailing whitespace
#         return match.group(1).strip()
    
#     # If 'ANSWER:' is present, extract from it
#     pattern_answer = r'^ANSWER:\s*(.*?)(?=\d+\.\s|$)'
#     match_answer = re.search(pattern_answer, text, re.DOTALL)
    
#     if match_answer:
#         return match_answer.group(1).strip()
#     elif text.startswith("สรุป:"):
#         return text[len("สรุป:"):].strip()
    
#     return None

def extract_answer(text):
    # Regular expression to capture everything before 'Assistant:', '**END OF TASK**', or end of text
    pattern = r'(.*?)(?=Assistant:|\*\*END OF TASK\*\*|$)'
    
    # Search for the pattern
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        text_before_specials = match.group(1).strip()
        
        # Remove leading labels like 'ANSWER:', 'สรุป:'
        text_cleaned = re.sub(r'^(ANSWER:|สรุป:)\s*', '', text_before_specials, flags=re.DOTALL).strip()
        
        return text_cleaned
    
    return None


# Extract the 'text' field from each dictionary
extracted_results = [extract(text['generated_output']) for text in sorted_results]
extracted_remove = [extract_answer(text) for text in extracted_results]




In [88]:
extracted_remove

['แผนการสื่อสาร ESG ของบริษัทซีพี จำกัด (มหาชน) หรือ CP กำหนดจะสื่อสารกับกลุ่มคนรุ่นใหม่อายุไม่เกิน 30 ปีในไตรมาสสุดท้ายของปี 2565 และยังคงสร้างการรับรู้ในกลุ่มผู้มีส่วนได้เสียผ่านช่องทางออนไลน์หลัก. CP ได้รับการยอมรับในฐานะองค์กรที่ยั่งยืนระดับโลกจาก Dow Jones Sustainability Index อย่างต่อเนื่อง 6 ปี และยังคงอยู่ใน Thaifex Sustainability Index ตลอด 5 ปีที่ผ่านมา. รวมถึงการชนะ 5 รางวัลในงาน Asia Excellence Awards รอบ 12 และได้รับรางวัลองค์กรที่สนับสนุนคนพิการเด่น 6 ปีต่อเนื่อง. ส่วน 7-Eleven ได้รับรางวัล Asian Energy Award ปี 2022 จากการประหยัดพลังงาน พร้อมทั้งร่วมกับ 437 โรงเรียนในเขตกรุงเทพฯ ในโครงการโรงเรียนไร้ขยะ ชุมชนยั่งยืน. CP ยังร่วมกับองค์กรไทยต่างชาติ AI จัดค่าย Creative AI Camp และยังร่วมกับหอการค้าไทยในการยกระดับ SMEs ผ่านโครงการ Business Accelerator และ Big Brother Season 6. นอกจากนี้ CP ยังช่วย SMEs ขายสินค้าใน 7-Eleven มากกว่า 8,000 รายการและเตรียมเพิ่ม 10% ในปี 2566 พร้อมสนับสนุนชุมชนผ่านกิจกรรมที่ส่งเสริมการอ่าน การเขียน และการสนับสนุนชุมชน.',
 'การประชุมผู้ถือหุ้นสามั

In [89]:
# Display the results
for idx, result in enumerate(extracted_remove):
    print(f"{result}")

แผนการสื่อสาร ESG ของบริษัทซีพี จำกัด (มหาชน) หรือ CP กำหนดจะสื่อสารกับกลุ่มคนรุ่นใหม่อายุไม่เกิน 30 ปีในไตรมาสสุดท้ายของปี 2565 และยังคงสร้างการรับรู้ในกลุ่มผู้มีส่วนได้เสียผ่านช่องทางออนไลน์หลัก. CP ได้รับการยอมรับในฐานะองค์กรที่ยั่งยืนระดับโลกจาก Dow Jones Sustainability Index อย่างต่อเนื่อง 6 ปี และยังคงอยู่ใน Thaifex Sustainability Index ตลอด 5 ปีที่ผ่านมา. รวมถึงการชนะ 5 รางวัลในงาน Asia Excellence Awards รอบ 12 และได้รับรางวัลองค์กรที่สนับสนุนคนพิการเด่น 6 ปีต่อเนื่อง. ส่วน 7-Eleven ได้รับรางวัล Asian Energy Award ปี 2022 จากการประหยัดพลังงาน พร้อมทั้งร่วมกับ 437 โรงเรียนในเขตกรุงเทพฯ ในโครงการโรงเรียนไร้ขยะ ชุมชนยั่งยืน. CP ยังร่วมกับองค์กรไทยต่างชาติ AI จัดค่าย Creative AI Camp และยังร่วมกับหอการค้าไทยในการยกระดับ SMEs ผ่านโครงการ Business Accelerator และ Big Brother Season 6. นอกจากนี้ CP ยังช่วย SMEs ขายสินค้าใน 7-Eleven มากกว่า 8,000 รายการและเตรียมเพิ่ม 10% ในปี 2566 พร้อมสนับสนุนชุมชนผ่านกิจกรรมที่ส่งเสริมการอ่าน การเขียน และการสนับสนุนชุมชน.
การประชุมผู้ถือหุ้นสามัญประจำ

In [91]:
extracted_remove

['แผนการสื่อสาร ESG ของบริษัทซีพี จำกัด (มหาชน) หรือ CP กำหนดจะสื่อสารกับกลุ่มคนรุ่นใหม่อายุไม่เกิน 30 ปีในไตรมาสสุดท้ายของปี 2565 และยังคงสร้างการรับรู้ในกลุ่มผู้มีส่วนได้เสียผ่านช่องทางออนไลน์หลัก. CP ได้รับการยอมรับในฐานะองค์กรที่ยั่งยืนระดับโลกจาก Dow Jones Sustainability Index อย่างต่อเนื่อง 6 ปี และยังคงอยู่ใน Thaifex Sustainability Index ตลอด 5 ปีที่ผ่านมา. รวมถึงการชนะ 5 รางวัลในงาน Asia Excellence Awards รอบ 12 และได้รับรางวัลองค์กรที่สนับสนุนคนพิการเด่น 6 ปีต่อเนื่อง. ส่วน 7-Eleven ได้รับรางวัล Asian Energy Award ปี 2022 จากการประหยัดพลังงาน พร้อมทั้งร่วมกับ 437 โรงเรียนในเขตกรุงเทพฯ ในโครงการโรงเรียนไร้ขยะ ชุมชนยั่งยืน. CP ยังร่วมกับองค์กรไทยต่างชาติ AI จัดค่าย Creative AI Camp และยังร่วมกับหอการค้าไทยในการยกระดับ SMEs ผ่านโครงการ Business Accelerator และ Big Brother Season 6. นอกจากนี้ CP ยังช่วย SMEs ขายสินค้าใน 7-Eleven มากกว่า 8,000 รายการและเตรียมเพิ่ม 10% ในปี 2566 พร้อมสนับสนุนชุมชนผ่านกิจกรรมที่ส่งเสริมการอ่าน การเขียน และการสนับสนุนชุมชน.',
 'การประชุมผู้ถือหุ้นสามั

In [28]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

2024-09-09 16:08:12,509 - INFO - PyTorch version 2.3.0 available.
2024-09-09 16:08:13,335 - INFO - Use pytorch device_name: cpu
2024-09-09 16:08:13,336 - INFO - Load pretrained SentenceTransformer: paraphrase-multilingual-MiniLM-L12-v2
c:\Users\kantaphong\anaconda3\envs\llm-env2\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:

# def check_similarity_with_original(chunks, cleaned_responses, model, threshold=0.7):
#     validated_responses = []
    
#     for i, (chunk, response) in enumerate(zip(chunks, cleaned_responses)):
#         # Encode both the original chunk and the cleaned response
#         chunk_embedding = model.encode(chunk, convert_to_tensor=True)
#         response_embedding = model.encode(response, convert_to_tensor=True)
        
#         # Calculate cosine similarity
#         cosine_score = util.pytorch_cos_sim(chunk_embedding, response_embedding).item()
        
#         # Print the similarity score and decide if it's validated
#         if cosine_score >= threshold:
#             print(f"Response {i+1} passed similarity check with a score of {cosine_score:.2f}")
#             validated_responses.append(response)
#         else:
#             print(f"Response {i+1} failed similarity check with a score of {cosine_score:.2f}")
#             validated_responses.append("Summary could not be validated.")
        
#         print("\n--------------------------------------------------\n")
    
#     return validated_responses

# # Example usage
# validated_responses = check_similarity_with_original(chunks, extracted_results, model)
